Imports

In [1]:
import cv2 as cv 
import os
import import_ipynb
import poseaux 

importing Jupyter notebook from poseaux.ipynb
importing Jupyter notebook from lateral_raise.ipynb
importing Jupyter notebook from print_result.ipynb


In [3]:
# Specify the paths for the 2 files
protoFile = "openpose_pose_mpi_faster_4_stages.prototxt"
weightsFile = "pose_iter_160000.caffemodel"
# Read the network into Memory
net = cv.dnn.readNetFromCaffe(protoFile, weightsFile)

# Minimium to mark point on result
threshold = 0.2
  
# Specify the input image dimensions
inWidth = 368
inHeight = 368

In [14]:
#for frame identity
index = 0

#Final Video name
pathOut = 'video_final.mp4'

frame_array = []

#Define the max angle of arms
angle_max_arms = 95

if not os.path.exists('./Result'):
    os.makedirs('./Result')

# set video file path of input video with name and extension
vid = cv.VideoCapture('../frame_frame/Abdominal/IMG_4165.mp4')
fps = vid.get(cv.CAP_PROP_FPS)
Height = vid.get(cv.CAP_PROP_FRAME_HEIGHT)
Width = vid.get(cv.CAP_PROP_FRAME_WIDTH)
size = (Width,Height)

while(True):        
    # Extract images
    ret, frame = vid.read()
    # end of frames
    if not ret: 
        break
        
    print ('Creating... Frame '+ str(index))
     
    # Prepare the frame to be fed to the network
    inpBlob = cv.dnn.blobFromImage(
        frame, 1.0 / 255, (inWidth, inHeight), (0, 0, 0), swapRB=False, crop=False)

    # Set the prepared object as the input blob of the network
    net.setInput(inpBlob)

    output = net.forward()

    H = output.shape[2]
    W = output.shape[3]
    # Empty list to store the detected keypoints
    points = []
    for i in range(15):
        # confidence map of corresponding body's part.
        probMap = output[0, i, :, :]

        # Find global maxima of the probMap.
        minVal, prob, minLoc, point = cv.minMaxLoc(probMap)
        Height, Width = frame.shape[:2]
        # Scale the point to fit on the original image
        x = (Width * point[0]) / W
        y = (Height * point[1]) / H

        if prob > threshold:
            cv.circle(frame, (int(x), int(y)), 15, (0, 255, 255), thickness=-1, lineType=cv.FILLED)
            cv.putText(frame, "{}".format(i), (int(x), int(y)), cv.FONT_HERSHEY_SIMPLEX, 
                           1.4, (0, 0, 255), 3, lineType=cv.LINE_AA)

            # Add the point to the list if the probability is greater than the threshold
            points.append((int(x), int(y)))
        else:
            points.append(None)

    poseaux.inspec_lateral_raise(frame, points, angle_max_arms, True)

    cv.imwrite("./Result/frame_r_"+str(index)+".jpg", frame)

    #inserting the frames into an image array
    frame_array.append(frame)
    
    # next frame
    index += 1
    
# Create Final Video
out = cv.VideoWriter(pathOut,cv.VideoWriter_fourcc(*'MP4V'), fps, size)
for i in range(len(frame_array)):
    # writing to a image array
    out.write(frame_array[i])
    
out.release()

print("DONE")

Creating... Frame 0
Problem in RWrist
Creating... Frame 1
Problem in RWrist
Creating... Frame 2
Problem in RWrist
Creating... Frame 3
Creating... Frame 4


KeyboardInterrupt: 